In [1]:
import math

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import pandas as pd

from scipy.optimize import curve_fit

mpl.use("nbagg")

# %matplotlib inline
# %matplotlib nbagg

C:\Anaconda2\lib\site-packages\matplotlib\__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
class volume_delay_equation:
    def __init__(self, name=""):
        self.name = name
    def __str__(self):
        print '''Volume Delay Equation: {0}
        '''.format(self.name)

In [3]:
class bpr_volume_delay_equation(volume_delay_equation):
    def __init__(self, a=0.15, b=4, name="BPR"):
            self.a = a
            self.b = b
            self.name = name
    def __str__(self):
        return(
            '''Volume Delay Equation: {0}
                 Parameter a = {1:.4f}
                 Parameter b = {2:.4f}
        '''.format(self.name, self.a, self.b)
        )
    
    def evaluate(self, qr):
        vr = 1 + self.a*(qr**self.b)
        return(vr)
    
    def equation(self):
        def f(x, a, b):
            return(1+a*(x**b))
        return(f)
    
    def estimate(self, qr, vr):
        # Estimate the parameters using least-square
        func = self.equation()
        popt, _ = curve_fit(func, qr, vr)
        self.a = popt[0]
        self.b = popt[1]

In [4]:
class conic_volume_delay_equation(volume_delay_equation):
    def __init__(self, a=10, name="Conic"):
        self.a = a
        self.b = (2.0*self.a-1.0)/(2.0*self.a-2.0)
        self.name = name
    def __str__(self):
        return(
            '''Volume Delay Equation: {0}
                 Parameter a = {1:.4f}
                 Parameter b = {2:.4f}
        '''.format(self.name, self.a, self.b)
        )
    
    def evaluate(self, qr):
        vr = 2+np.sqrt(self.a**2*(1-qr)**2+self.b**2)-self.a*(1-qr)-self.b
        return(vr)
    
    def equation(self):
        def f(x, a):
            return(2+np.sqrt(a**2*(1-x)**2+((2.0*a-1.0)/(2.0*a-2.0))**2)-a*(1-x)-((2.0*a-1.0)/(2.0*a-2.0)))
        return(f)
    
    def estimate(self, qr, vr):
        # Estimating the parameters using least-square
        func = self.equation()
        popt, _ = curve_fit(func, qr, vr, p0 = 8.0)
        self.a = popt[0]
        self.b = (2.0*self.a-1.0)/(2.0*self.a-2.0)

In [5]:
def plot_vdf(qr, vr, labels, title=""):
    _, axes = plt.subplots(1,1, figsize=(16,12))
    if (vr.ndim == 1):
        axes.scatter(qr, vr, label = labels, marker="o", s=2)
    else:
        for k in range(0, vr.shape[0]):
            axes.scatter(qr[k], vr[k], label = labels[k], marker="o", s=4)
    
    axes.set_xlim([0,2.5])
    axes.set_ylim([1,10])
    axes.set_xlabel("Flow Ratio/Density Ratio")
    axes.set_ylabel("Speed Ratio")
    axes.legend(loc='upper left', prop={'size': 14})
    axes.grid(True)
    axes.set_title(title)
    
    for item in ([axes.title, axes.xaxis.label, axes.yaxis.label] + axes.get_xticklabels() + axes.get_yticklabels()):
        item.set_fontsize(14)
    
    return axes

In [6]:
def plot_estimation(ax, qr, vr, labels, title=""):
    # tr needs to be a 2-element list. one for observed, one for predictions    
    # _, axes = plt.subplots(1,1, figsize=(12,8))
    
    for k in range(0, len(vr)):
        ax.scatter(qr[k], vr[k], label = labels[k], marker="o", s=4)
        
    ax.set_xlim([0,2.5])
    ax.set_ylim([1,10])
    ax.set_xlabel("Flow Ratio/Density Ratio")
    ax.set_ylabel("Speed Ratio")
    ax.legend(loc='upper left', prop={'size': 11})
    ax.grid(True)
    ax.set_title(title)

In [7]:
def plot_estimation_grid(qrs, vrs, labels, titles):
    _, axes = plt.subplots(len(qrs)/2,len(qrs)/2, figsize=(16,12))
    axes = axes.flatten()
    k = 0
    for q, v, l, t in zip(qrs, vrs, labels, titles):
        plot_estimation(axes[k], q, v, l, t)
        k += 1

In [8]:
def get_rmse(y, y_pred):
    return np.sqrt(np.mean((y - y_pred)**2))

In [9]:
def report_rmse(obs, est, labels):
    for y, y_pred, l in zip(obs, est, labels):
        print("RMSE for Model {0} = {1:.2f}".format(l, get_rmse(y, y_pred)) )

### Estimation Using VISSIM Data 

In [98]:
MPR = str(100)
FT = "MINOR"
BPR_MODEL_FIG = "FIGURES\BPR_"+FT+"_"+"M"+MPR+".png"
BPR_MODEL_EVAL_FIG = "FIGURES\BPR_MODEL_EVAL_"+"M"+MPR+"_"+FT+".png"

CONIC_MODEL_FIG = "FIGURES\CONIC_"+FT+"_"+"M"+MPR+".png"
CONIC_MODEL_EVAL_FIG = "FIGURES\CONIC_MODEL_EVAL_"+"M"+MPR+"_"+FT+".png"

DATA_FILE = "Link_Seg_Data_"+FT+"_M"+MPR+".csv"
PARA_FILE = "spd_cal_"+FT+"_M"+MPR+"_adv.out"
LINK_SELECTION_FREEWAY = [10,11,17]
LINK_SELECTION_MAJOR = [78, 81, 82, 83, 101, 102] 
LINK_SELECTION_MINOR = [108, 110, 111, 114, 124, 130]

LINK_SELECTION = LINK_SELECTION_MINOR

In [99]:
data_mpr = pd.read_csv(DATA_FILE)
params_mpr = pd.read_csv(PARA_FILE, sep="\s*", skiprows=5, engine='python')
data_mpr_CRITICAL_DENSITY = params_mpr.loc[:,"Cap"][0] / (params_mpr.loc[:,"SC"][0] / 1.6)
data_mpr_FF_SPD = params_mpr.loc[:,"FS"][0] / 1.6
data_mpr_CRITICAL_SPD = params_mpr.loc[:,"SC"][0] / 1.6
data_mpr_CAP = params_mpr.loc[:,"Cap"][0]

In [100]:
# Compute the DENSITYRATIO, TIMERATIO
data_mpr["DENSITY_RATIO"] = data_mpr.LANEDENSITY/data_mpr_CRITICAL_DENSITY
data_mpr["TIME_RATIO"] = data_mpr_FF_SPD / data_mpr.SPEED
data_mpr["VOLUME_RATIO"] = data_mpr.LANEFLOW / data_mpr_CAP
data_mpr["VOLUME_RATIO_2"] = data_mpr.VOLUME_RATIO
data_mpr.loc[data_mpr.SPEED < data_mpr_CRITICAL_SPD,  "VOLUME_RATIO_2"] = (2*data_mpr_CAP-data_mpr.LANEFLOW) / data_mpr_CAP

In [101]:
kr = data_mpr.loc[data_mpr.LINK.isin(LINK_SELECTION) & (data_mpr.SPEED > 0), "DENSITY_RATIO"].values
tr = data_mpr.loc[data_mpr.LINK.isin(LINK_SELECTION) & (data_mpr.SPEED > 0), "TIME_RATIO"].values
fr = data_mpr.loc[data_mpr.LINK.isin(LINK_SELECTION) & (data_mpr.SPEED > 0), "VOLUME_RATIO"].values
fr2 = data_mpr.loc[data_mpr.LINK.isin(LINK_SELECTION) & (data_mpr.SPEED > 0), "VOLUME_RATIO_2"].values

In [102]:
pd.DataFrame(np.vstack([kr, tr, fr, fr2]).T, columns=["kr", "tr", "fr", "fr_transformed"]).describe()

,kr,tr,fr,fr_transformed
count,436918.000000,436918.000000,436918.000000,436918.000000
mean,0.293532,1.104286,0.303543,0.355009
std,0.423681,1.026900,0.340921,0.430425
min,0.000125,0.957866,0.000085,0.000085
25%,0.022295,1.001623,0.026671,0.026671
50%,0.053434,1.007154,0.062231,0.066677
75%,0.506443,1.013002,0.578534,0.620958
max,6.327419,108.445946,1.283042,1.999237


### BPR Function

In [103]:
bpr_std = bpr_volume_delay_equation(a=0.15, b=4, name="Standard BPR")
bpr_fr = bpr_volume_delay_equation(name="Flow Ratio")
bpr_fr2 = bpr_volume_delay_equation(name="Transformed Flow Ratio")
bpr_kr = bpr_volume_delay_equation(name="Density Ratio")

bpr_kr.estimate(kr, tr)
bpr_fr.estimate(fr, tr)
bpr_fr2.estimate(fr2, tr)

tr_std = bpr_std.evaluate(kr)
tr_kr = bpr_kr.evaluate(kr)
tr_fr = bpr_fr.evaluate(kr)
tr_fr2 = bpr_fr2.evaluate(kr)

print(bpr_std)
print(bpr_fr)
print(bpr_fr2)
print(bpr_kr)

h = plot_vdf([kr, kr, kr, kr],np.array([tr_kr, tr_fr, tr_fr2, tr_std]),
             [bpr_kr.name, bpr_fr.name, bpr_fr2.name, bpr_std.name], 
             "BPR Function - FREEWAY - MPR = 0%")
plt.savefig(BPR_MODEL_FIG)

Volume Delay Equation: Standard BPR
                 Parameter a = 0.1500
                 Parameter b = 4.0000
        
Volume Delay Equation: Flow Ratio
                 Parameter a = 0.1502
                 Parameter b = 0.1761
        
Volume Delay Equation: Transformed Flow Ratio
                 Parameter a = 0.3943
                 Parameter b = 2.8341
        
Volume Delay Equation: Density Ratio
                 Parameter a = 0.4446
                 Parameter b = 2.2596
        


In [104]:
report_rmse([tr_kr, tr_fr, tr_fr2, tr_std], [tr]*4, [bpr_kr.name, bpr_fr.name, bpr_fr2.name, bpr_std.name])

RMSE for Model Density Ratio = 0.69
RMSE for Model Flow Ratio = 1.02
RMSE for Model Transformed Flow Ratio = 1.05
RMSE for Model Standard BPR = 2.76


In [105]:
plot_estimation_grid([[kr, kr], [fr, fr], [fr2, fr2], [kr, kr]], 
                     [[tr, tr_kr], [tr, tr_fr], [tr, tr_fr2], [tr, tr_std]], 
                     [("Observed", "Predicted")]*4, 
                     ["Density Ratio", "Flow Ratio", "Transformed Flow Ratio", "Standard BPR"])
plt.savefig(BPR_MODEL_EVAL_FIG)

### Conic Function

In [106]:
conic_std = conic_volume_delay_equation(a=15, name="Alpha = 15")
conic_fr  = conic_volume_delay_equation(name="Flow Ratio")
conic_fr2 = conic_volume_delay_equation(name="Transformed Flow Ratio")
conic_kr  = conic_volume_delay_equation(name="Density Ratio")

conic_kr.estimate(kr, tr)
conic_fr.estimate(fr, tr)
conic_fr2.estimate(fr2, tr)

conic_tr_std = conic_std.evaluate(kr)
conic_tr_kr  = conic_kr.evaluate(kr)
conic_tr_fr  = conic_fr.evaluate(kr)
conic_tr_fr2 = conic_fr2.evaluate(kr)

print(conic_std)
print(conic_fr)
print(conic_fr2)
print(conic_kr)

h = plot_vdf([kr, kr, kr, kr],np.array([conic_tr_kr, conic_tr_fr, conic_tr_fr2, conic_tr_std]),
             [conic_kr.name, conic_fr.name, conic_fr2.name, conic_std.name], "Conic Function - FREEWAY - MPR = 0%")
plt.savefig(CONIC_MODEL_FIG)

Volume Delay Equation: Alpha = 15
                 Parameter a = 15.0000
                 Parameter b = 1.0357
        
Volume Delay Equation: Flow Ratio
                 Parameter a = 3.1343
                 Parameter b = 1.2343
        
Volume Delay Equation: Transformed Flow Ratio
                 Parameter a = 1.6891
                 Parameter b = 1.7256
        
Volume Delay Equation: Density Ratio
                 Parameter a = 1.8417
                 Parameter b = 1.5940
        


In [107]:
report_rmse([conic_tr_kr, conic_tr_fr, conic_tr_fr2, conic_tr_std], [tr]*4, 
            [conic_kr.name, conic_fr.name, conic_fr2.name, conic_std.name])

RMSE for Model Density Ratio = 0.74
RMSE for Model Flow Ratio = 0.96
RMSE for Model Transformed Flow Ratio = 0.74
RMSE for Model Alpha = 15 = 6.12


In [108]:
plot_estimation_grid([[kr, kr], [fr, fr], [fr2, fr2], [kr, kr]], 
                     [[tr, conic_tr_kr], [tr, conic_tr_fr], [tr, conic_tr_fr2], [tr, conic_tr_std]], 
                     [("Observed", "Predicted")]*4, 
                     ["Density Ratio", "Flow Ratio", "Transformed Flow Ratio", "Standard BPR"])
plt.savefig(CONIC_MODEL_EVAL_FIG)